All necessary libraries for this assignment have already been added. You are not allowed to add any additional imports.

In [1]:
%pip install datasets
%pip install nltk
# !pip install datasets
# !pip install nltk

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import regex as re

import pandas as pd
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import nltk
from datasets import load_dataset

nltk.download("punkt")


[nltk_data] Downloading package punkt to /Users/macbook/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# code here
# Bernoulli Naive Bayes
golf_data = pd.read_csv('/Users/macbook/Desktop/semester 5/ML/PA1/golf_data.csv')  # Replace with correct path

In [4]:
import pandas as pd
from datasets import load_dataset

emotion_labels = {
    0: 'anger',
    1: 'joy',
    2: 'optimism',
    3: 'sadness'
}

tweet_data = load_dataset('tweet_eval', 'emotion', cache_dir="datasets")

def convert_to_dataframe(split):
    texts = []
    labels = []
    
    for entry in tweet_data[split]:
        text = entry['text']
        label = entry['label']
        
        texts.append(text)
        labels.append(label)
    
    df = pd.DataFrame({'text': texts, 'label': labels})
    return df

train_df = convert_to_dataframe('train')
val_df = convert_to_dataframe('validation')
test_df = convert_to_dataframe('test')

print(train_df.head())


Generating train split:   0%|          | 0/3257 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1421 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/374 [00:00<?, ? examples/s]

                                                text  label
0  “Worry is a down payment on a problem you may ...      2
1  My roommate: it's okay that we can't spell bec...      0
2  No but that's so cute. Atsu was probably shy a...      1
3  Rooneys fucking untouchable isn't he? Been fuc...      0
4  it's pretty depressing when u hit pan on ur fa...      3


##### Before proceeding with further tasks, ensure you have determined which type of Naive Bayes is most suitable for each dataset.

## 2. Data Preprocessing

### 2.1 Preprocessing the Golf Dataset

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split

file_path = '/Users/macbook/Desktop/semester 5/ML/PA1/golf_data.csv'
golf_data = pd.read_csv(file_path)

print("Initial Golf Dataset:")
print(golf_data.head())

month_map = {'Winter': 1, 'Non-Winter': 0}
golf_data['Month'] = golf_data['Month'].map(month_map)

season_map = {'Winter': 1, 'Non-Winter': 0}
golf_data['Season'] = golf_data['Season'].map(season_map)

golf_data.rename(columns={'Month': 'month_winter', 'Season': 'season_winter'}, inplace=True)

temperature_map = {'high': 1, 'low': 0}
golf_data['Temperature'] = golf_data['Temperature'].map(temperature_map)

humidity_map = {'high': 1, 'low': 0}
golf_data['Humidity'] = golf_data['Humidity'].map(humidity_map)

golf_data.rename(columns={'Temperature': 'temperature_high', 'Humidity': 'humidity_high'}, inplace=True)

golf_data.rename(columns={'Windy': 'windy'}, inplace=True)

outlook_map = {'sunny': 1, 'not sunny': 0}
golf_data['Outlook'] = golf_data['Outlook'].map(outlook_map)

golf_data.rename(columns={'Outlook': 'outlook_sunny'}, inplace=True)

crowdedness_map = {'high': 1, 'not high': 0}
golf_data['Crowdedness'] = golf_data['Crowdedness'].map(crowdedness_map)

golf_data.rename(columns={'Crowdedness': 'crowded_high'}, inplace=True)

print("\nEncoded Golf Dataset:")
print(golf_data.head())

X = golf_data.drop('Play', axis=1)
y = golf_data['Play']

test_size = 0.3
random_state = 42

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

print("\nTraining set shape:", X_train.shape, y_train.shape)
print("Test set shape:", X_test.shape, y_test.shape)

# golf_data.to_csv('/Users/macbook/Desktop/semester 5/ML/PA1/golf_data_encoded.csv', index=False)
# print("\nEncoded Golf Dataset saved to 'golf_data_encoded.csv'")


Initial Golf Dataset:
   Holiday   Month  Season Temperature Humidity  Windy Outlook Crowdedness  \
0        1  Winter  Winter         low      low      1   sunny        high   
1        1  Winter  Winter         low      low      1   sunny        high   
2        1  Winter  Winter         low      low      1   sunny        high   
3        1  Winter  Winter         low      low      1   sunny        high   
4        1  Winter  Winter         low      low      1   sunny        high   

   Play  
0     1  
1     0  
2     0  
3     1  
4     1  

Encoded Golf Dataset:
   Holiday  month_winter  season_winter  temperature_high  humidity_high  \
0        1             1              1                 0              0   
1        1             1              1                 0              0   
2        1             1              1                 0              0   
3        1             1              1                 0              0   
4        1             1              1       

### 2.2 Preprocessing the Tweet Eval Dataset

In [6]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

print("Downloading NLTK resources...")
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    url_pattern = r'http\S+|www\S+|https\S+'
    text = re.sub(url_pattern, '', text, flags=re.MULTILINE)
    
    mention_pattern = r'@\w+'
    text = re.sub(mention_pattern, '', text)
    
    non_alphanumeric_pattern = r'[^a-zA-Z0-9\s]'
    text = re.sub(non_alphanumeric_pattern, '', text)
    
    text = text.lower()
    
    whitespace_pattern = r'\s+'
    text = re.sub(whitespace_pattern, ' ', text)
    text = text.strip()
    
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    text = ' '.join(filtered_words)
    
    lemmatized_words = [lemmatizer.lemmatize(word) for word in text.split()]
    text = ' '.join(lemmatized_words)
    
    return text

print("Cleaning training data...")
train_df['cleaned_text'] = train_df['text'].apply(clean_text)

print("Cleaning validation data...")
val_df['cleaned_text'] = val_df['text'].apply(clean_text)

print("Cleaning test data...")
test_df['cleaned_text'] = test_df['text'].apply(clean_text)

print("\nOriginal vs Cleaned Text in Training Data:")
print(train_df[['text', 'cleaned_text']].head())

print("\nColumn names in Training DataFrame:")
print(train_df.columns)


Cleaning training data...


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/macbook/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/macbook/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Cleaning validation data...
Cleaning test data...

Original vs Cleaned Text in Training Data:
                                                text  \
0  “Worry is a down payment on a problem you may ...   
1  My roommate: it's okay that we can't spell bec...   
2  No but that's so cute. Atsu was probably shy a...   
3  Rooneys fucking untouchable isn't he? Been fuc...   
4  it's pretty depressing when u hit pan on ur fa...   

                                        cleaned_text  
0  worry payment problem may never joyce meyer mo...  
1  roommate okay cant spell autocorrect terrible ...  
2  thats cute atsu probably shy photo cherry help...  
3  rooneys fucking untouchable isnt fucking dread...  
4  pretty depressing u hit pan ur favourite highl...  

Column names in Training DataFrame:
Index(['text', 'label', 'cleaned_text'], dtype='object')


## 3. Implementing Naive Bayes from Scratch

In [7]:
import numpy as np

class BernoulliNaiveBayes:
    def __init__(self):
        self.class_priors = None
        self.conditional_probs = None
        self.classes = None
    
    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.classes = np.unique(y)
        n_classes = len(self.classes)

        self.class_priors = np.zeros(n_classes)
        self.conditional_probs = np.zeros((n_classes, n_features))

        for idx, c in enumerate(self.classes):
            X_c = X[y == c]
            self.class_priors[idx] = X_c.shape[0] / n_samples
            numerator = X_c.sum(axis=0) + 1
            denominator = X_c.shape[0] + 2
            self.conditional_probs[idx, :] = numerator / denominator

    def predict(self, X):
        log_class_priors = np.log(self.class_priors)
        log_conditional_probs = np.log(self.conditional_probs.T)
        log_complementary_probs = np.log(1 - self.conditional_probs.T)
        log_probs = log_class_priors + X @ log_conditional_probs + (1 - X) @ log_complementary_probs
        predicted_classes = np.argmax(log_probs, axis=1)
        return self.classes[predicted_classes]


In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

bnb_model = BernoulliNaiveBayes()
bnb_model.fit(X_train.values, y_train.values)

y_pred_val = bnb_model.predict(X_test.values)

accuracy_val = accuracy_score(y_test.values, y_pred_val)
precision_val = precision_score(y_test.values, y_pred_val, average='weighted')
recall_val = recall_score(y_test.values, y_pred_val, average='weighted')
f1_val = f1_score(y_test.values, y_pred_val, average='weighted')
confusion_val = confusion_matrix(y_test.values, y_pred_val)

print("\n")
print("Bernoulli Naive Bayes for Test Dataset - Golf\n")
print(f"Accuracy: {accuracy_val:.4f}")
print(f"Precision: {precision_val:.4f}")
print(f"Recall: {recall_val:.4f}")
print(f"F1 Score: {f1_val:.4f}")
print("Confusion Matrix:")
print(confusion_val)




Bernoulli Naive Bayes for Test Dataset - Golf

Accuracy: 0.8200
Precision: 0.7685
Recall: 0.8200
F1 Score: 0.7494
Confusion Matrix:
[[1872   13]
 [ 401   14]]


## 3.2 Multinomial Naive Bayes (Manual Implementation)

In [9]:
import numpy as np

class BagOfWords:
    def __init__(self):
        self.vocabulary = {}
        self.word_to_index = {}

    def fit(self, corpus):
        unique_words = set()
        for sentence in corpus:
            words_in_sentence = sentence.split()
            unique_words.update(words_in_sentence)
        
        self.vocabulary = sorted(unique_words)
        self.word_to_index = {word: idx for idx, word in enumerate(self.vocabulary)}
    
    def vectorize(self, sentence):
        vector_length = len(self.vocabulary)
        vector = np.zeros(vector_length, dtype=int)
        words_in_sentence = sentence.split()
        
        for word in words_in_sentence:
            if word in self.word_to_index:
                index = self.word_to_index[word]
                vector[index] += 1
        
        return vector


In [10]:
# code here
vocabulary = ["the", "cat", "sat", "on", "mat"]

word_to_index = {word: idx for idx, word in enumerate(vocabulary)}

vector = np.zeros(len(vocabulary), dtype=int)

sentence = "the cat sat on the mat"

for word in sentence.split():
    if word in word_to_index:
        index = word_to_index[word]
        vector[index] += 1

print(f"Sentence: {sentence}")
print(f"Vocabulary: {vocabulary}")
print(f"Word-to-index mapping: {word_to_index}")
print(f"Vectorized form: {vector}")


Sentence: the cat sat on the mat
Vocabulary: ['the', 'cat', 'sat', 'on', 'mat']
Word-to-index mapping: {'the': 0, 'cat': 1, 'sat': 2, 'on': 3, 'mat': 4}
Vectorized form: [2 1 1 1 1]


In [11]:
import numpy as np

class NaiveBayes:
    def __init__(self):
        self.logprior = None
        self.loglikelihood = None
        self.vocabulary = None
        self.classes = None

    def train(self, X, y):
        num_docs = len(y)
        self.classes = np.unique(y)
        num_classes = len(self.classes)
        self.logprior = np.zeros(num_classes)
        self.loglikelihood = np.zeros((X.shape[1], num_classes))
        self.vocabulary = X.shape[1]
        
        for idx, c in enumerate(self.classes):
            Nc = np.sum(y == c)
            self.logprior[idx] = np.log(Nc) - np.log(num_docs)
            bigdoc_c = X[y == c]
            word_counts = np.sum(bigdoc_c, axis=0)
            total_count_c = np.sum(word_counts)
            
            for w in range(X.shape[1]):
                word_prob = (word_counts[w] + 1) / (total_count_c + self.vocabulary)
                self.loglikelihood[w, idx] = np.log(word_prob)

    def predict(self, X):
        num_docs = X.shape[0]
        scores = np.zeros((num_docs, len(self.classes)))

        for c_idx, c in enumerate(self.classes):
            scores[:, c_idx] = self.logprior[c_idx] + np.dot(X, self.loglikelihood[:, c_idx])

        predicted_classes = self.classes[np.argmax(scores, axis=1)]
        return predicted_classes


In [12]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

vectorizer = CountVectorizer()
train_vectors = vectorizer.fit_transform(train_df['cleaned_text']).toarray()
val_vectors = vectorizer.transform(val_df['cleaned_text']).toarray()
test_vectors = vectorizer.transform(test_df['cleaned_text']).toarray()

nb_model = NaiveBayes()
nb_model.train(train_vectors, train_df['label'].values)

val_predictions = nb_model.predict(val_vectors)
accuracy_val = accuracy_score(val_df['label'].values, val_predictions)
precision_val = precision_score(val_df['label'].values, val_predictions, average='weighted')
recall_val = recall_score(val_df['label'].values, val_predictions, average='weighted')
f1_val = f1_score(val_df['label'].values, val_predictions, average='weighted')
confusion_val = confusion_matrix(val_df['label'].values, val_predictions)

print("\nNaive Bayes for Twitter Dataset - Validation Results\n")
print(f"Accuracy: {accuracy_val:.4f}")
print(f"Precision: {precision_val:.4f}")
print(f"Recall: {recall_val:.4f}")
print(f"F1 Score: {f1_val:.4f}")
print("Confusion Matrix:")
print(confusion_val)

test_predictions = nb_model.predict(test_vectors)
accuracy_test = accuracy_score(test_df['label'].values, test_predictions)
precision_test = precision_score(test_df['label'].values, test_predictions, average='weighted')
recall_test = recall_score(test_df['label'].values, test_predictions, average='weighted')
f1_test = f1_score(test_df['label'].values, test_predictions, average='weighted')
confusion_test = confusion_matrix(test_df['label'].values, test_predictions)

print("\nNaive Bayes for Twitter Dataset - Test Results\n")
print(f"Accuracy: {accuracy_test:.4f}")
print(f"Precision: {precision_test:.4f}")
print(f"Recall: {recall_test:.4f}")
print(f"F1 Score: {f1_test:.4f}")
print("Confusion Matrix:")
print(confusion_test)



Naive Bayes for Twitter Dataset - Validation Results

Accuracy: 0.6791
Precision: 0.6888
Recall: 0.6791
F1 Score: 0.6565
Confusion Matrix:
[[145   5   1   9]
 [ 30  50   1  16]
 [ 13   4   4   7]
 [ 29   5   0  55]]

Naive Bayes for Twitter Dataset - Test Results

Accuracy: 0.6615
Precision: 0.6726
Recall: 0.6615
F1 Score: 0.6398
Confusion Matrix:
[[493  22   2  41]
 [107 185   5  61]
 [ 67  13  18  25]
 [114  20   4 244]]


## 4. Implementing Naive Bayes using sklearn



In [13]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

nb_model_sklearn = MultinomialNB()
nb_model_sklearn.fit(train_vectors, train_df['label'].values)

val_predictions_sklearn = nb_model_sklearn.predict(val_vectors)

accuracy_sklearn = accuracy_score(val_df['label'].values, val_predictions_sklearn)
precision_sklearn = precision_score(val_df['label'].values, val_predictions_sklearn, average='weighted')
recall_sklearn = recall_score(val_df['label'].values, val_predictions_sklearn, average='weighted')
f1_sklearn = f1_score(val_df['label'].values, val_predictions_sklearn, average='weighted')
confusion_sklearn = confusion_matrix(val_df['label'].values, val_predictions_sklearn)

print("\n")
print("Multinomial Naive Bayes for Validation Dataset - TWITTER DATASET\n")
print(f"Accuracy (sklearn): {accuracy_sklearn:.4f}")
print(f"Precision (sklearn): {precision_sklearn:.4f}")
print(f"Recall (sklearn): {recall_sklearn:.4f}")
print(f"F1 Score (sklearn): {f1_sklearn:.4f}")
print("Confusion Matrix (sklearn):")
print(confusion_sklearn)

test_predictions_sklearn = nb_model_sklearn.predict(test_vectors)

accuracy_test = accuracy_score(test_df['label'].values, test_predictions_sklearn)
precision_test = precision_score(test_df['label'].values, test_predictions_sklearn, average='weighted')
recall_test = recall_score(test_df['label'].values, test_predictions_sklearn, average='weighted')
f1_test = f1_score(test_df['label'].values, test_predictions_sklearn, average='weighted')
confusion_test = confusion_matrix(test_df['label'].values, test_predictions_sklearn)

print("\n")
print("Multinomial Naive Bayes for Test Dataset - TWITTER DATASET\n")
print(f"Accuracy (sklearn): {accuracy_test:.4f}")
print(f"Precision (sklearn): {precision_test:.4f}")
print(f"Recall (sklearn): {recall_test:.4f}")
print(f"F1 Score (sklearn): {f1_test:.4f}")
print("Confusion Matrix (sklearn):")
print(confusion_test)




Multinomial Naive Bayes for Validation Dataset - TWITTER DATASET

Accuracy (sklearn): 0.6791
Precision (sklearn): 0.6888
Recall (sklearn): 0.6791
F1 Score (sklearn): 0.6565
Confusion Matrix (sklearn):
[[145   5   1   9]
 [ 30  50   1  16]
 [ 13   4   4   7]
 [ 29   5   0  55]]


Multinomial Naive Bayes for Test Dataset - TWITTER DATASET

Accuracy (sklearn): 0.6615
Precision (sklearn): 0.6726
Recall (sklearn): 0.6615
F1 Score (sklearn): 0.6398
Confusion Matrix (sklearn):
[[493  22   2  41]
 [107 185   5  61]
 [ 67  13  18  25]
 [114  20   4 244]]


In [14]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

bernoulli_nb_model = BernoulliNB()

bernoulli_nb_model.fit(X_train, y_train)

y_pred = bernoulli_nb_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
confusion = confusion_matrix(y_test, y_pred)

print("\n")
print("Bernoulli Naive Bayes - GOLF DATASET\n")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("Confusion Matrix:")
print(confusion)




Bernoulli Naive Bayes - GOLF DATASET

Accuracy: 0.8200
Precision: 0.7685
Recall: 0.8200
F1 Score: 0.7494
Confusion Matrix:
[[1872   13]
 [ 401   14]]


## 5. Conclusion


I selected Multinomial Naive Bayes for the Twitter sentiment analysis dataset because it required classifying into four distinct sentiment groups: anger, joy, optimism, and sadness. This algorithm is effective for multi-class problems since it considers the frequency of words within a document to make predictions. This feature is useful for distinguishing between sentiments like joy and optimism, which are both positive emotions. The frequency of words can provide context allowing for more accurate distinctions among these categories. Given that tweets can often be lengthy the occurrences of each word can significantly influence the overall tone and help identify the dominant sentiment. If the task had only involved classifying into positive versus negative emotions, Bernoulli Naive Bayes might have been enough.

On the other hand I chose Bernoulli Naive Bayes for the golf dataset because it was designed for making binary predictions: whether or not a person would play golf based on certain factors. The factors in this dataset were also binary (e.g., windy or not windy) so tracking the frequency of word occurrences was unnecessary.